<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [1]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [2]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [3]:
def x1(x, v):
  return math.sin(math.log(abs(v[0])) / x[0])

def x2(x, v):
  return math.cos(v[1]**3)

def x3(x, v):
  return math.sin(math.tan(x[0] / x[1]) - 3**(v[2]))

def v1(x, v):
  return (v[0] + math.tan(x[0])**2)

def v2(x, v):
  return (0.2 * v[1])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * v[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[1])))

def y3(x, v):
  return v[2] * x[0] / (x[1] + 0.1)

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [4]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [5]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_3 = createTableDNM(functions, count, 3, np.array(model_2))
model_4 = createTableDNM(functions, count, 4, np.array(model_3))
model_5 = createTableDNM(functions, count, 5, np.array(model_4))
model_6 = createTableDNM(functions, count, 6, np.array(model_5))
model_7 = createTableDNM(functions, count, 7, np.array(model_6))
model_8 = createTableDNM(functions, count, 8, np.array(model_7))
model_9 = createTableDNM(functions, count, 9, np.array(model_8))
model_10 = createTableDNM(functions, count, 10, np.array(model_9))

In [6]:
model_9

,x1(9),x2(9),x3(9),v1(9),v2(9),v3(9),y1(9),y2(9),y3(9)
0,-0.961356,1.000000,-0.961767,1.258846,-0.267236,-0.060490,-0.748779,0.513874,0.052866
1,0.894285,0.999731,0.983080,0.912067,-0.169424,5.324242,0.927229,0.410627,4.329594
2,0.993040,0.984337,-0.251075,1.250025,-0.057772,1.947912,0.791647,0.240292,1.783906
3,-0.253641,0.996926,0.976743,3.622801,0.468596,1.110991,-0.035260,0.672037,-0.256893
4,0.994745,0.997278,0.232220,-0.772624,0.097951,1.241242,-1.284102,0.312721,1.125257
...,...,...,...,...,...,...,...,...,...
95,0.726808,0.999923,-0.299036,472.917348,-0.294520,11.262990,0.001310,0.538777,7.442366
96,0.995794,0.988474,-0.991169,29.759003,-0.515093,-0.591639,0.033391,0.701866,-0.541262
97,-0.853736,0.997032,0.609300,0.874469,0.182182,-0.781412,-0.902071,0.425647,0.608113
98,-0.751906,0.992669,-0.136284,36.053753,0.034531,5.585219,-0.018084,0.185806,-3.843398


In [7]:
model_10

,x1(10),x2(10),x3(10),v1(10),v2(10),v3(10),y1(10),y2(10),y3(10)
0,-0.237167,0.999818,-0.698572,15.595209,-0.377982,2.145724,-0.007406,0.607491,-0.462708
1,-0.102740,0.999988,-0.161637,-0.591834,-0.003180,2.151284,0.055643,0.056394,-0.200933
2,0.222840,1.000000,-0.587633,2.512576,0.153480,0.208339,0.041867,0.390997,0.042206
3,0.934948,0.994711,0.486003,-0.747432,0.589807,1.877778,-1.209509,0.745789,1.603733
4,-0.256428,1.000000,-0.703222,5.554201,0.001319,0.675305,-0.023379,0.036311,-0.157425
...,...,...,...,...,...,...,...,...,...
95,0.813909,0.999674,-0.784712,-0.753750,0.296640,18.671963,-0.974174,0.540655,13.819808
96,-0.262751,0.990676,0.872852,0.086088,-0.462823,0.997311,-1.564490,0.668189,-0.240259
97,0.156473,0.999982,-0.999983,-1.131543,0.280126,0.386194,-0.054700,0.525810,0.054936
98,0.998460,1.000000,0.970712,-1.871739,0.557402,0.927170,-0.533029,0.727312,0.841583


##1. Линейная форма функции пересчёта

In [8]:
#Построение линейной регрессионной модели
def buildLinearRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  estimate_linear = []
  variance_linear = []
  error_linear = []

  linearFunction = (model1.copy())[cols[0:(len(cols) - 1)]]
  linearFunction[cols[len(cols) - 1]] = (model2.copy())[cols[len(cols) - 1]]
  linearFunction.insert(0, "optional_col", vector_ones)

  A_linear = np.array(linearFunction)
  #A_psev = BenIsrael(A_linear[:, 0:(len(A_linear[0]) - 1)])
  X_linear = solvingSLAE(A_linear[:, 0:(len(A_linear[0]) - 1)], np.matrix(A_linear[:, len(A_linear[0]) - 1]).T)

  for i in range(len(linearFunction)):
    for j in range(0, len(X_linear)):
      if(j == 0):
        estimate_linear.append(X_linear[j,0])
      else:
        estimate_linear[i] += X_linear[j,0] * np.array(linearFunction)[i,j]

    variance_linear.append((1.0 / len(linearFunction)) * (np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])**2)
    #error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            #/ np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1]) * 100)
    error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            / max(np.array(linearFunction)[:, len(np.array(linearFunction)[0]) - 1])) * 100)

  linearFunction['regression'] = estimate_linear
  linearFunction['variance'] = variance_linear
  linearFunction['error, %'] = error_linear

  return X_linear, linearFunction

###Для функции $x_1[t+1]$

In [9]:
#x1, v1+
X1_linear, X1_linearTable = buildLinearRegretion(model_9, model_10, ['x1(9)', 'v1(9)', 'x1(10)'])
print(pd.DataFrame(X1_linear, columns=['X1'], index=['a1', 'c11', 'b11']))
X1_linearTable

               X1
a1   6.791714e-02
c11  2.144816e-01
b11 -6.448275e-07


,optional_col,x1(9),v1(9),x1(10),regression,variance,"error, %"
0,1,-0.961356,1.258846,-0.237167,-0.138277,0.000098,0.099043
1,1,0.894285,0.912067,-0.102740,0.259724,0.001314,0.363024
2,1,0.993040,1.250025,0.222840,0.280905,0.000034,0.058154
3,1,-0.253641,3.622801,0.934948,0.013513,0.008490,0.922856
4,1,0.994745,-0.772624,-0.256428,0.281272,0.002891,0.538530
...,...,...,...,...,...,...,...
95,1,0.726808,472.917348,0.813909,0.223499,0.003486,0.591321
96,1,0.995794,29.759003,-0.262751,0.281477,0.002962,0.545068
97,1,-0.853736,0.874469,0.156473,-0.115194,0.000738,0.272086
98,1,-0.751906,36.053753,0.998460,-0.093376,0.011921,1.093520


###Для функции $x_2[t+1]$

In [10]:
#v2+
X2_linear, X2_linearTable = buildLinearRegretion(model_9, model_10, ['v2(9)', 'x2(10)'])
print(pd.DataFrame(X2_linear, columns=['X2'], index=['a2', 'b22']))
X2_linearTable

           X2
a2   0.996722
b22  0.003482


,optional_col,v2(9),x2(10),regression,variance,"error, %"
0,1,-0.267236,0.999818,0.995791,1.621207e-07,0.004026
1,1,-0.169424,0.999988,0.996132,1.486965e-07,0.003856
2,1,-0.057772,1.000000,0.996521,1.210449e-07,0.003479
3,1,0.468596,0.994711,0.998354,1.326945e-07,0.003643
4,1,0.097951,1.000000,0.997063,8.623006e-08,0.002936
...,...,...,...,...,...,...
95,1,-0.294520,0.999674,0.995696,1.581826e-07,0.003977
96,1,-0.515093,0.990676,0.994928,1.808426e-07,0.004253
97,1,0.182182,0.999982,0.997356,6.892529e-08,0.002625
98,1,0.034531,1.000000,0.996842,9.971495e-08,0.003158


###Для функции $x_3[t+1]$

In [11]:
#x1, x2, v3+
X3_linear, X3_linearTable = buildLinearRegretion(model_9, model_10, ['x1(9)', 'x2(9)', 'v3(9)', 'x3(10)'])
print(pd.DataFrame(X3_linear, columns=['X3'], index=['a3', 'c31', 'c32', 'b33']))
X3_linearTable

            X3
a3   21.820814
c31  -0.115194
c32 -22.074003
b33   0.008221


,optional_col,x1(9),x2(9),v3(9),x3(10),regression,variance,"error, %"
0,1,-0.961356,1.000000,-0.060490,-0.698572,-0.142937,0.003087,0.555819
1,1,0.894285,0.999731,5.324242,-0.161637,-0.306505,0.000210,0.144916
2,1,0.993040,0.984337,1.947912,-0.587633,-0.005812,0.003385,0.582014
3,1,-0.253641,0.996926,1.110991,0.486003,-0.146972,0.004007,0.633186
4,1,0.994745,0.997278,1.241242,-0.703222,-0.297486,0.001646,0.405871
...,...,...,...,...,...,...,...,...
95,1,0.726808,0.999923,11.262990,-0.784712,-0.242632,0.002939,0.542260
96,1,0.995794,0.988474,-0.591639,0.872852,-0.118330,0.009824,0.991511
97,1,-0.853736,0.997032,-0.781412,-0.999983,-0.095753,0.008176,0.904530
98,1,-0.751906,0.992669,5.585219,0.970712,0.041172,0.008640,0.929849


###Для функции $y_1[t]$

In [12]:
#x1, v1+
Y1_linear, Y1_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'v1(9)', 'y1(9)'])
print(pd.DataFrame(Y1_linear, columns=['Y1'], index=['a1', 'c11', 'b11']))
Y1_linearTable

               Y1
a1  -3.193742e-02
c11  1.437722e-01
b11 -5.249048e-08


,optional_col,x1(9),v1(9),y1(9),regression,variance,"error, %"
0,1,-0.961356,1.258846,-0.748779,-0.170154,0.003348,0.187340
1,1,0.894285,0.912067,0.927229,0.096636,0.006899,0.268918
2,1,0.993040,1.250025,0.791647,0.110834,0.004635,0.220425
3,1,-0.253641,3.622801,-0.035260,-0.068404,0.000011,0.010731
4,1,0.994745,-0.772624,-1.284102,0.111079,0.019465,0.451713
...,...,...,...,...,...,...,...
95,1,0.726808,472.917348,0.001310,0.072533,0.000051,0.023059
96,1,0.995794,29.759003,0.033391,0.111228,0.000061,0.025201
97,1,-0.853736,0.874469,-0.902071,-0.154681,0.005586,0.241980
98,1,-0.751906,36.053753,-0.018084,-0.140043,0.000149,0.039486


###Для функции $y_2[t]$

In [13]:
#v2+
Y2_linear, Y2_linearTable = buildLinearRegretion(model_9, model_9, ['v2(9)', 'y2(9)'])
print(pd.DataFrame(Y2_linear, columns=['Y2'], index=['a2', 'b22']))
Y2_linearTable

           Y2
a2   0.508749
b22 -0.004797


,optional_col,v2(9),y2(9),regression,variance,"error, %"
0,1,-0.267236,0.513874,0.510031,1.476952e-07,0.005148
1,1,-0.169424,0.410627,0.509562,9.788057e-05,0.132515
2,1,-0.057772,0.240292,0.509026,7.221798e-04,0.359947
3,1,0.468596,0.672037,0.506501,2.740231e-04,0.221723
4,1,0.097951,0.312721,0.508279,3.824289e-04,0.261934
...,...,...,...,...,...,...
95,1,-0.294520,0.538777,0.510162,8.188494e-06,0.038328
96,1,-0.515093,0.701866,0.511220,3.634601e-04,0.255355
97,1,0.182182,0.425647,0.507875,6.761376e-05,0.110137
98,1,0.034531,0.185806,0.508583,1.041851e-03,0.432334


###Для функции $y_3[t]$

In [14]:
#x1, x2, v3+
Y3_linear, Y3_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'x2(9)', 'v3(9)', 'y3(9)'])
print(pd.DataFrame(Y3_linear, columns=['Y3'], index=['a3', 'c31', 'c32', 'b33']))
Y3_linearTable

            Y3
a3  -42.640620
c31   2.761003
c32  42.198701
b33   0.175881


,optional_col,x1(9),x2(9),v3(9),y3(9),regression,variance,"error, %"
0,1,-0.961356,1.000000,-0.060490,0.052866,-3.106878,0.099840,0.181209
1,1,0.894285,0.999731,5.324242,4.329594,2.952297,0.018969,0.078987
2,1,0.993040,0.984337,1.947912,1.783906,1.981489,0.000390,0.011331
3,1,-0.253641,0.996926,1.110991,-0.256893,-1.076559,0.006719,0.047007
4,1,0.994745,0.997278,1.241242,1.125257,2.408017,0.016455,0.073565
...,...,...,...,...,...,...,...,...
95,1,0.726808,0.999923,11.262990,7.442366,3.542511,0.152089,0.223654
96,1,0.995794,0.988474,-0.591639,-0.541262,1.717015,0.050998,0.129510
97,1,-0.853736,0.997032,-0.781412,0.608113,-3.061765,0.134680,0.210465
98,1,-0.751906,0.992669,5.585219,-3.843398,-1.844973,0.039937,0.114608


####Матрица ошибок и отклонений

In [15]:
errorsTable = pd.DataFrame([[ sum(X1_linearTable['variance']), sum(X2_linearTable['variance']), sum(X3_linearTable['variance']),
                              sum(Y1_linearTable['variance']), sum(Y2_linearTable['variance']), sum(Y3_linearTable['variance'])],
                            [ sum(X1_linearTable['error, %']), sum(X2_linearTable['error, %']), sum(X3_linearTable['error, %']),
                              sum(Y1_linearTable['error, %']), sum(Y2_linearTable['error, %']), sum(Y3_linearTable['error, %'])]],
                           index=['E', 'A, %'], columns=['x1', 'x2', 'x3', 'y1', 'y2', 'y3'])
errorsTable

,x1,x2,x3,y1,y2,y3
E,0.464379,0.000029,0.465463,0.376577,0.026428,9.304227
"A, %",60.488492,0.402816,59.909389,11.924940,19.058101,12.392586


##2. Квадратичная форма функции пересчёта

In [16]:
#Построение квадратичной регрессионной модели
def buildQuadraticRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  estimate = []
  variance = []
  error = []

  #переменные
  Function = (model1.copy())[cols[0:(len(cols) - 1)]]
  i = 0
  name_col = ''
  while(i < len(cols) - 1):
    temporary_cols = []
    j = i

    while(j < len(cols) - 1):
      name_col = cols[i] + ' * ' + cols[j]
      for k in range(len(model1)):
        temporary_cols.append(Function[cols[i]][k] * Function[cols[j]][k])

      Function[name_col] = temporary_cols
      temporary_cols.clear()
      j += 1

    i += 1

  #значение функции
  Function[cols[len(cols) - 1]] = (model2.copy())[cols[len(cols) - 1]]

  #доп. столбец
  Function.insert(0, "optional_col", vector_ones)

  A = np.array(Function)
  #A_psev = BenIsrael(A[:, 0:(len(A[0]) - 1)])
  X = solvingSLAE(A[:, 0:(len(A[0]) - 1)], np.matrix(A[:, len(A[0]) - 1]).T)

  for i in range(len(Function)):
    for j in range(0, len(X)):
      if(j == 0):
        estimate.append(X[j,0])
      else:
        estimate[i] += X[j,0] * np.array(Function)[i,j]

    variance.append((1.0 / len(Function)) * (np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])**2)
    #error_linear.append((1.0 / len(Function)) * abs((np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])
                            #/ np.array(Function)[i, len(np.array(Function)[0]) - 1]) * 100)
    error.append((1.0 / len(Function)) * abs((np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])
                            / max(np.array(Function)[:, len(np.array(Function)[0]) - 1])) * 100)

  Function['regression'] = estimate
  Function['variance'] = variance
  Function['error, %'] = error

  return X, Function

###Для функции $x_1[t + 1]$

In [17]:
#x1, v1+
X1_quadratic, X1_quadraticTable = buildQuadraticRegretion(model_9, model_10, ['x1(9)', 'v1(9)', 'x1(10)'])
print(pd.DataFrame(X1_quadratic, columns=['X1'], index=['a1', 'c11', 'b11', 'd111', 'd112', 'd113']))
X1_quadraticTable

                X1
a1    2.516999e-01
c11   2.285369e-01
b11  -1.701638e-04
d111 -3.574233e-01
d113  2.229330e-04
d112 -1.134456e-11


,optional_col,x1(9),v1(9),x1(9) * x1(9),x1(9) * v1(9),v1(9) * v1(9),x1(10),regression,variance,"error, %"
0,1,-0.961356,1.258846,0.924205,-1.210199,1.584693,-0.237167,-0.298822,0.000038,0.061750
1,1,0.894285,0.912067,0.799746,0.815648,0.831867,-0.102740,0.170256,0.000745,0.273417
2,1,0.993040,1.250025,0.986129,1.241325,1.562561,0.222840,0.126245,0.000093,0.096745
3,1,-0.253641,3.622801,0.064334,-0.918891,13.124686,0.934948,0.169918,0.005853,0.766210
4,1,0.994745,-0.772624,0.989518,-0.768564,0.596948,-0.256428,0.125319,0.001457,0.382336
...,...,...,...,...,...,...,...,...,...,...
95,1,0.726808,472.917348,0.528250,343.720180,223650.818373,0.813909,0.225144,0.003466,0.589673
96,1,0.995794,29.759003,0.991605,29.633827,885.598288,-0.262751,0.126395,0.001514,0.389746
97,1,-0.853736,0.874469,0.728865,-0.746566,0.764697,0.156473,-0.204239,0.001301,0.361268
98,1,-0.751906,36.053753,0.565363,-27.109040,1299.873078,0.998460,-0.134391,0.012834,1.134598


###Для функции $x_2[t+1]$

In [18]:
#v2+
X2_quadratic, X2_quadraticTable = buildQuadraticRegretion(model_9, model_10, ['v2(9)', 'x2(10)'])
print(pd.DataFrame(X2_quadratic, columns=['X2'], index=['a2', 'b22', 'd221']))
X2_quadraticTable

            X2
a2    1.001949
b22   0.002355
d221 -0.045243


,optional_col,v2(9),v2(9) * v2(9),x2(10),regression,variance,"error, %"
0,1,-0.267236,0.071415,0.999818,0.998089,2.989972e-08,0.001729
1,1,-0.169424,0.028704,0.999988,1.000251,6.929302e-10,0.000263
2,1,-0.057772,0.003338,1.000000,1.001662,2.762259e-08,0.001662
3,1,0.468596,0.219582,0.994711,0.993118,2.537873e-08,0.001593
4,1,0.097951,0.009594,1.000000,1.001746,3.048636e-08,0.001746
...,...,...,...,...,...,...,...
95,1,-0.294520,0.086742,0.999674,0.997331,5.487971e-08,0.002343
96,1,-0.515093,0.265321,0.990676,0.988732,3.777761e-08,0.001944
97,1,0.182182,0.033190,0.999982,1.000876,8.004302e-09,0.000895
98,1,0.034531,0.001192,1.000000,1.001976,3.906122e-08,0.001976


###Для функции $x_3[t+1]$

In [19]:
#x1, x2, v3+
X3_quadratic, X3_quadraticTable = buildQuadraticRegretion(model_9, model_10, ['x1(9)', 'x2(9)', 'v3(9)', 'x3(10)'])
print(pd.DataFrame(X3_quadratic, columns=['X3'], index=['a3', 'c31', 'c32', 'b33', 'd331', 'd332', 'd333', 'd334', 'd335', 'd336',]))
X3_quadraticTable

               X3
a3     683.492952
c31    -46.886454
c32  -1337.943938
b33     -0.283771
d331     0.278586
d332    46.988159
d333    -0.047519
d334   653.997525
d335     0.302460
d336     0.000054


,optional_col,x1(9),x2(9),v3(9),x1(9) * x1(9),x1(9) * x2(9),x1(9) * v3(9),x2(9) * x2(9),x2(9) * v3(9),v3(9) * v3(9),x3(10),regression,variance,"error, %"
0,1,-0.961356,1.000000,-0.060490,0.924205,-0.961356,0.058153,0.999999,-0.060490,0.003659,-0.698572,-0.297636,0.001607,0.401069
1,1,0.894285,0.999731,5.324242,0.799746,0.894045,4.761390,0.999463,5.322811,28.347552,-0.161637,-0.268575,0.000114,0.106974
2,1,0.993040,0.984337,1.947912,0.986129,0.977486,1.934355,0.968918,1.917401,3.794360,-0.587633,-0.243600,0.001184,0.344147
3,1,-0.253641,0.996926,1.110991,0.064334,-0.252861,-0.281793,0.993861,1.107575,1.234300,0.486003,-0.293249,0.006072,0.779510
4,1,0.994745,0.997278,1.241242,0.989518,0.992038,1.234720,0.994563,1.237863,1.540682,-0.703222,-0.153904,0.003018,0.549500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0.726808,0.999923,11.262990,0.528250,0.726752,8.186033,0.999847,11.262128,126.854955,-0.784712,-0.404663,0.001444,0.380176
96,1,0.995794,0.988474,-0.591639,0.991605,0.984316,-0.589150,0.977080,-0.584819,0.350036,0.872852,-0.164150,0.010754,1.037347
97,1,-0.853736,0.997032,-0.781412,0.728865,-0.851202,0.667120,0.994073,-0.779093,0.610605,-0.999983,-0.169100,0.006904,0.831159
98,1,-0.751906,0.992669,5.585219,0.565363,-0.746394,-4.199561,0.985391,5.544272,31.194674,0.970712,0.434528,0.002875,0.536363


###Для функции $y_1[t]$

In [20]:
#x1, v1+
Y1_quadratic, Y1_quadraticTable = buildQuadraticRegretion(model_9, model_9, ['x1(9)', 'v1(9)', 'y1(9)'])
print(pd.DataFrame(Y1_quadratic, columns=['Y1'], index=['a1', 'c11', 'b11', 'd111', 'd112', 'd113']))
Y1_quadraticTable

                Y1
a1    1.091864e-01
c11   1.875093e-01
b11   1.022250e-04
d111 -2.924867e-01
d112 -1.273405e-04
d113  3.446958e-12


,optional_col,x1(9),v1(9),x1(9) * x1(9),x1(9) * v1(9),v1(9) * v1(9),y1(9),regression,variance,"error, %"
0,1,-0.961356,1.258846,0.924205,-1.210199,1.584693,-0.748779,-0.341112,0.001662,0.131989
1,1,0.894285,0.912067,0.799746,0.815648,0.831867,0.927229,0.042947,0.007820,0.286301
2,1,0.993040,1.250025,0.986129,1.241325,1.562561,0.791647,0.006931,0.006158,0.254065
3,1,-0.253641,3.622801,0.064334,-0.918891,13.124686,-0.035260,0.043297,0.000062,0.025434
4,1,0.994745,-0.772624,0.989518,-0.768564,0.596948,-1.284102,0.006308,0.016652,0.417792
...,...,...,...,...,...,...,...,...,...,...
95,1,0.726808,472.917348,0.528250,343.720180,223650.818373,0.001310,0.095539,0.000089,0.030508
96,1,0.995794,29.759003,0.991605,29.633827,885.598288,0.033391,0.005144,0.000008,0.009146
97,1,-0.853736,0.874469,0.728865,-0.746566,0.764697,-0.902071,-0.263896,0.004073,0.206620
98,1,-0.751906,36.053753,0.565363,-27.109040,1299.873078,-0.018084,-0.190026,0.000296,0.055669


###Для функции $y_2[t]$

In [21]:
#v2+
Y2_quadratic, Y2_quadraticTable = buildQuadraticRegretion(model_9, model_9, ['v2(9)', 'y2(9)'])
print(pd.DataFrame(Y2_quadratic, columns=['Y2'], index=['a2', 'b22', 'd221']))
Y2_quadraticTable

            Y2
a2    0.346663
b22   0.030161
d221  1.402950


,optional_col,v2(9),v2(9) * v2(9),y2(9),regression,variance,"error, %"
0,1,-0.267236,0.071415,0.513874,0.438794,5.636943e-05,0.100563
1,1,-0.169424,0.028704,0.410627,0.381824,8.296306e-06,0.038580
2,1,-0.057772,0.003338,0.240292,0.349603,1.194886e-04,0.146413
3,1,0.468596,0.219582,0.672037,0.668859,1.010288e-07,0.004257
4,1,0.097951,0.009594,0.312721,0.363077,2.535760e-05,0.067448
...,...,...,...,...,...,...,...
95,1,-0.294520,0.086742,0.538777,0.459475,6.288878e-05,0.106219
96,1,-0.515093,0.265321,0.701866,0.703359,2.229484e-08,0.002000
97,1,0.182182,0.033190,0.425647,0.398722,7.249972e-06,0.036065
98,1,0.034531,0.001192,0.185806,0.349377,2.675544e-04,0.219090


###Для функции $y_3[t]$

In [22]:
#x1, x2, v3+
Y3_quadratic, Y3_quadraticTable = buildQuadraticRegretion(model_9, model_9, ['x1(9)', 'x2(9)', 'v3(9)', 'y3(9)'])
print(pd.DataFrame(Y3_quadratic, columns=['Y3'], index=['a3', 'c31', 'c32', 'b33', 'd331', 'd332', 'd333', 'd334', 'd335', 'd336']))
Y3_quadraticTable

              Y3
a3     56.768302
c31     1.556715
c32  -114.985240
b33     0.114362
d331   -0.003124
d332   -1.558540
d333    0.910154
d334   58.219983
d335   -0.115092
d336    0.000041


,optional_col,x1(9),x2(9),v3(9),x1(9) * x1(9),x1(9) * x2(9),x1(9) * v3(9),x2(9) * x2(9),x2(9) * v3(9),v3(9) * v3(9),y3(9),regression,variance,"error, %"
0,1,-0.961356,1.000000,-0.060490,0.924205,-0.961356,0.058153,0.999999,-0.060490,0.003659,0.052866,0.054884,4.071215e-08,0.000116
1,1,0.894285,0.999731,5.324242,0.799746,0.894045,4.761390,0.999463,5.322811,28.347552,4.329594,4.329945,1.228886e-09,0.000020
2,1,0.993040,0.984337,1.947912,0.986129,0.977486,1.934355,0.968918,1.917401,3.794360,1.783906,1.776699,5.193863e-07,0.000413
3,1,-0.253641,0.996926,1.110991,0.064334,-0.252861,-0.281793,0.993861,1.107575,1.234300,-0.256893,-0.258672,3.163770e-08,0.000102
4,1,0.994745,0.997278,1.241242,0.989518,0.992038,1.234720,0.994563,1.237863,1.540682,1.125257,1.122162,9.580205e-08,0.000178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0.726808,0.999923,11.262990,0.528250,0.726752,8.186033,0.999847,11.262128,126.854955,7.442366,7.447680,2.823246e-07,0.000305
96,1,0.995794,0.988474,-0.591639,0.991605,0.984316,-0.589150,0.977080,-0.584819,0.350036,-0.541262,-0.529571,1.366988e-06,0.000671
97,1,-0.853736,0.997032,-0.781412,0.728865,-0.851202,0.667120,0.994073,-0.779093,0.610605,0.608113,0.602082,3.636786e-07,0.000346
98,1,-0.751906,0.992669,5.585219,0.565363,-0.746394,-4.199561,0.985391,5.544272,31.194674,-3.843398,-3.833808,9.197228e-07,0.000550


####Матрица ошибок и отклонений

In [23]:
errorsTable2 = pd.DataFrame([[sum(X1_quadraticTable['variance']), sum(X2_quadraticTable['variance']), sum(X3_quadraticTable['variance']),
                              sum(Y1_quadraticTable['variance']), sum(Y2_quadraticTable['variance']), sum(Y3_quadraticTable['variance'])],
                            [ sum(X1_quadraticTable['error, %']), sum(X2_quadraticTable['error, %']), sum(X3_quadraticTable['error, %']),
                              sum(Y1_quadraticTable['error, %']), sum(Y2_quadraticTable['error, %']), sum(Y3_quadraticTable['error, %'])]],
                           index=['E', 'A, %'], columns=['x1', 'x2', 'x3', 'y1', 'y2', 'y3'])
errorsTable2

,x1,x2,x3,y1,y2,y3
E,0.438109,0.000005,0.418795,0.366297,0.003756,0.000039
"A, %",57.955105,0.184715,54.686581,12.208192,6.474582,0.021769


In [24]:
errorsTable

,x1,x2,x3,y1,y2,y3
E,0.464379,0.000029,0.465463,0.376577,0.026428,9.304227
"A, %",60.488492,0.402816,59.909389,11.924940,19.058101,12.392586
